In [1]:
import pandas as pd
from gprofiler import GProfiler
gp = GProfiler(return_dataframe=True)

In [2]:
delta_correlation_df = pd.read_csv('delta_correlation_df.csv')
delta_correlation_df

,Gene,Delta_Correlation,P_Value,FDR,Cancer
0,A1BG,-0.268533,5.703182e-02,1.320375e-01,CCRCC
1,A1CF,0.192038,1.063340e-04,6.401858e-04,CCRCC
2,A2M,-0.191619,1.277644e-01,2.439276e-01,CCRCC
3,AAAS,0.019654,8.963138e-01,9.409267e-01,CCRCC
4,AACS,-0.169937,6.007042e-02,1.375402e-01,CCRCC
...,...,...,...,...,...
50303,ZXDC,0.073854,6.395306e-01,6.962162e-01,LUAD
50304,ZYG11B,0.734569,1.583743e-10,1.504331e-09,LUAD
50305,ZYX,0.440861,2.253307e-06,9.499585e-06,LUAD
50306,ZZEF1,0.422001,3.731185e-11,3.889196e-10,LUAD


In [3]:
# 'CCRCC', 'Endometrial', 'HNSCC', 'LSCC', 'LUAD'
all_profiles = []
for cancer in pd.unique(delta_correlation_df.Cancer):
    cancer_df = delta_correlation_df[delta_correlation_df.Cancer == cancer]
    background_genes  = list(pd.unique(cancer_df.Gene))
    cancer_df = cancer_df[cancer_df.FDR < 0.05]
    cancer_profile = gp.profile(organism='hsapiens', query = background_genes, no_iea=True, sources = ["KEGG"],
                       ordered=True, no_evidences=False, background= background_genes)
    all_profiles.append(cancer_profile)

In [4]:
#CCRCC
(all_profiles[0])

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences
0,KEGG,KEGG:02010,ABC transporters,2.890010e-38,True,ABC transporters,24,39,20,9916,0.512821,0.833333,query_1,[KEGG:00000],"[ABCA1, ABCA12, ABCA2, ABCA6, ABCA8, ABCB1, AB...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
1,KEGG,KEGG:00071,Fatty acid degradation,6.087431e-26,True,Fatty acid degradation,37,327,25,9916,0.076453,0.675676,query_1,[KEGG:00000],"[ACAA1, ACAA2, ACADL, ACADM, ACADS, ACADSB, AC...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
2,KEGG,KEGG:01100,Metabolic pathways,4.844512e-20,True,Metabolic pathways,1077,374,127,9916,0.339572,0.117920,query_1,[KEGG:00000],"[AACS, AADAT, AASS, ABAT, ABO, ACAA1, ACAA2, A...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
3,KEGG,KEGG:01212,Fatty acid metabolism,1.189195e-17,True,Fatty acid metabolism,46,119,16,9916,0.134454,0.347826,query_1,[KEGG:00000],"[ACAA1, ACAA2, ACADL, ACADM, ACADS, ACADSB, AC...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
4,KEGG,KEGG:00620,Pyruvate metabolism,1.210257e-14,True,Pyruvate metabolism,41,327,18,9916,0.055046,0.439024,query_1,[KEGG:00000],"[ACACB, ACAT1, ACAT2, ACSS1, ACSS2, ACYP1, ACY...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,KEGG,KEGG:04925,Aldosterone synthesis and secretion,3.626267e-02,True,Aldosterone synthesis and secretion,49,1146,20,9916,0.017452,0.408163,query_1,[KEGG:00000],"[ADCY5, ADCY6, ADCY9, AGT, ATF1, ATF2, ATP1A1,...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
91,KEGG,KEGG:05133,Pertussis,3.873217e-02,True,Pertussis,49,1199,17,9916,0.014178,0.346939,query_1,[KEGG:00000],"[C1QA, C1QB, C1QC, C1R, C1S, C2, C3, C4A, C4B,...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
92,KEGG,KEGG:05132,Salmonella infection,4.280470e-02,True,Salmonella infection,201,141,9,9916,0.063830,0.044776,query_1,[KEGG:00000],"[ABI1, ACBD3, ACTB, ACTG1, ACTR10, ACTR1A, ACT...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
93,KEGG,KEGG:03018,RNA degradation,4.763459e-02,True,RNA degradation,61,2763,32,9916,0.011582,0.524590,query_1,[KEGG:00000],"[C1D, CNOT1, CNOT10, CNOT2, CNOT3, CNOT4, CNOT...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."


In [5]:
#Endometrial
(all_profiles[1])

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences
0,KEGG,KEGG:00071,Fatty acid degradation,6.210303e-25,True,Fatty acid degradation,34,288,24,9211,0.083333,0.705882,query_1,[KEGG:00000],"[ACAA1, ACAA2, ACADM, ACADS, ACADSB, ACADVL, A...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
1,KEGG,KEGG:02010,ABC transporters,2.733223e-24,True,ABC transporters,16,34,14,9211,0.411765,0.875000,query_1,[KEGG:00000],"[ABCA8, ABCB1, ABCB10, ABCB6, ABCB7, ABCB8, AB...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
2,KEGG,KEGG:01212,Fatty acid metabolism,4.427950e-18,True,Fatty acid metabolism,43,105,16,9211,0.152381,0.372093,query_1,[KEGG:00000],"[ACAA1, ACAA2, ACACA, ACADM, ACADS, ACADSB, AC...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
3,KEGG,KEGG:01100,Metabolic pathways,4.152109e-16,True,Metabolic pathways,971,331,114,9211,0.344411,0.117405,query_1,[KEGG:00000],"[A4GALT, AACS, AADAT, AASS, ABAT, ABHD14A-ACY1...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
4,KEGG,KEGG:00620,Pyruvate metabolism,4.376681e-15,True,Pyruvate metabolism,39,288,18,9211,0.062500,0.461538,query_1,[KEGG:00000],"[ACACA, ACAT1, ACAT2, ACSS1, ACSS2, ACYP1, ACY...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,KEGG,KEGG:03018,RNA degradation,2.927983e-02,True,RNA degradation,63,2600,33,9211,0.012692,0.523810,query_1,[KEGG:00000],"[C1D, CNOT1, CNOT10, CNOT2, CNOT3, CNOT4, CNOT...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
79,KEGG,KEGG:04810,Regulation of actin cytoskeleton,4.078564e-02,True,Regulation of actin cytoskeleton,153,125,8,9211,0.064000,0.052288,query_1,[KEGG:00000],"[ABI2, ACTB, ACTG1, ACTN1, ACTN4, ACTR2, ACTR3...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
80,KEGG,KEGG:05214,Glioma,4.248990e-02,True,Glioma,54,1339,19,9211,0.014190,0.351852,query_1,[KEGG:00000],"[AKT1, AKT2, AKT3, ARAF, BAK1, BAX, BRAF, CALM...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
81,KEGG,KEGG:04072,Phospholipase D signaling pathway,4.658734e-02,True,Phospholipase D signaling pathway,83,267,10,9211,0.037453,0.120482,query_1,[KEGG:00000],"[ADCY6, AGPAT1, AGPAT2, AGPAT3, AGPAT4, AGPAT5...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."


In [6]:
#HNSCC
(all_profiles[2])

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences
0,KEGG,KEGG:02010,ABC transporters,4.597778e-38,True,ABC transporters,23,40,20,10080,0.500000,0.869565,query_1,[KEGG:00000],"[ABCA1, ABCA12, ABCA6, ABCA8, ABCB1, ABCB10, A...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
1,KEGG,KEGG:00071,Fatty acid degradation,5.911467e-28,True,Fatty acid degradation,37,321,26,10080,0.080997,0.702703,query_1,[KEGG:00000],"[ACAA1, ACAA2, ACADL, ACADM, ACADS, ACADSB, AC...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
2,KEGG,KEGG:01212,Fatty acid metabolism,3.423311e-19,True,Fatty acid metabolism,48,121,17,10080,0.140496,0.354167,query_1,[KEGG:00000],"[ACAA1, ACAA2, ACACA, ACADL, ACADM, ACADS, ACA...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
3,KEGG,KEGG:00620,Pyruvate metabolism,1.495226e-17,True,Pyruvate metabolism,40,321,20,10080,0.062305,0.500000,query_1,[KEGG:00000],"[ACACA, ACACB, ACAT1, ACAT2, ACSS1, ACSS2, ACY...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
4,KEGG,KEGG:01100,Metabolic pathways,9.783940e-17,True,Metabolic pathways,1036,370,118,10080,0.318919,0.113900,query_1,[KEGG:00000],"[AACS, AASS, ABAT, ABO, ACAA1, ACAA2, ACACA, A...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,KEGG,KEGG:04923,Regulation of lipolysis in adipocytes,3.158984e-02,True,Regulation of lipolysis in adipocytes,28,299,6,10080,0.020067,0.214286,query_1,[KEGG:00000],"[ABHD5, ADCY3, ADCY7, AKT1, AKT2, AKT3]","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG]]"
95,KEGG,KEGG:04972,Pancreatic secretion,3.706223e-02,True,Pancreatic secretion,47,738,15,10080,0.020325,0.319149,query_1,[KEGG:00000],"[ADCY3, ADCY7, AMY1B, ATP1A1, ATP1A2, ATP1A3, ...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
96,KEGG,KEGG:05017,Spinocerebellar ataxia,3.920827e-02,True,Spinocerebellar ataxia,107,792,20,10080,0.025253,0.186916,query_1,[KEGG:00000],"[ADRM1, AFG3L2, AKT1, AKT2, AKT3, AMBRA1, ATG1...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
97,KEGG,KEGG:03010,Ribosome,4.021966e-02,True,Ribosome,127,7434,126,10080,0.016949,0.992126,query_1,[KEGG:00000],"[FAU, MRPL1, MRPL10, MRPL11, MRPL12, MRPL13, M...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."


In [7]:
#LSCC
(all_profiles[3])

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences
0,KEGG,KEGG:02010,ABC transporters,2.396327e-58,True,ABC transporters,31,48,28,10857,0.583333,0.903226,query_1,[KEGG:00000],"[ABCA1, ABCA12, ABCA13, ABCA2, ABCA3, ABCA6, A...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
1,KEGG,KEGG:00071,Fatty acid degradation,5.893935e-26,True,Fatty acid degradation,37,358,25,10857,0.069832,0.675676,query_1,[KEGG:00000],"[ACAA1, ACAA2, ACADL, ACADM, ACADS, ACADSB, AC...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
2,KEGG,KEGG:01212,Fatty acid metabolism,1.040404e-20,True,Fatty acid metabolism,48,133,18,10857,0.135338,0.375000,query_1,[KEGG:00000],"[ACAA1, ACAA2, ACACA, ACADL, ACADM, ACADS, ACA...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
3,KEGG,KEGG:01100,Metabolic pathways,3.511436e-20,True,Metabolic pathways,1071,414,129,10857,0.311594,0.120448,query_1,[KEGG:00000],"[A4GALT, AACS, AASS, ABAT, ABHD14A-ACY1, ABO, ...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
4,KEGG,KEGG:00620,Pyruvate metabolism,1.125799e-14,True,Pyruvate metabolism,39,358,18,10857,0.050279,0.461538,query_1,[KEGG:00000],"[ACACA, ACACB, ACAT1, ACAT2, ACSS1, ACSS2, ACY...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,KEGG,KEGG:04270,Vascular smooth muscle contraction,3.021247e-02,True,Vascular smooth muscle contraction,83,278,9,10857,0.032374,0.108434,query_1,[KEGG:00000],"[ACTA2, ACTG2, ADCY3, ADCY5, ADCY6, ADCY7, ADC...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
109,KEGG,KEGG:04261,Adrenergic signaling in cardiomyocytes,3.359283e-02,True,Adrenergic signaling in cardiomyocytes,87,1269,28,10857,0.022065,0.321839,query_1,[KEGG:00000],"[ADCY3, ADCY5, ADCY6, ADCY7, ADCY9, AGT, AKT1,...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
110,KEGG,KEGG:04725,Cholinergic synapse,3.459136e-02,True,Cholinergic synapse,62,335,9,10857,0.026866,0.145161,query_1,[KEGG:00000],"[ACHE, ADCY3, ADCY5, ADCY6, ADCY7, ADCY9, AKT1...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
111,KEGG,KEGG:01522,Endocrine resistance,3.793022e-02,True,Endocrine resistance,71,335,8,10857,0.023881,0.112676,query_1,[KEGG:00000],"[ADCY3, ADCY5, ADCY6, ADCY7, ADCY9, AKT1, AKT2...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."


In [8]:
#LUAD
(all_profiles[4])

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences
0,KEGG,KEGG:02010,ABC transporters,9.936227e-53,True,ABC transporters,29,45,26,9998,0.577778,0.896552,query_1,[KEGG:00000],"[ABCA1, ABCA12, ABCA13, ABCA2, ABCA3, ABCA6, A...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
1,KEGG,KEGG:00071,Fatty acid degradation,1.171066e-25,True,Fatty acid degradation,36,340,25,9998,0.073529,0.694444,query_1,[KEGG:00000],"[ACAA1, ACAA2, ACADL, ACADM, ACADS, ACADSB, AC...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
2,KEGG,KEGG:01212,Fatty acid metabolism,4.795055e-19,True,Fatty acid metabolism,45,124,17,9998,0.137097,0.377778,query_1,[KEGG:00000],"[ACAA1, ACAA2, ACACA, ACADL, ACADM, ACADS, ACA...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
3,KEGG,KEGG:01100,Metabolic pathways,2.255569e-17,True,Metabolic pathways,1041,389,124,9998,0.318766,0.119116,query_1,[KEGG:00000],"[AACS, AADAT, AASS, ABAT, ABO, ACAA1, ACAA2, A...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
4,KEGG,KEGG:00620,Pyruvate metabolism,1.823597e-14,True,Pyruvate metabolism,38,340,18,9998,0.052941,0.473684,query_1,[KEGG:00000],"[ACACA, ACAT1, ACAT2, ACSS1, ACSS2, ACYP1, ACY...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,KEGG,KEGG:04725,Cholinergic synapse,4.056759e-02,True,Cholinergic synapse,61,318,9,9998,0.028302,0.147541,query_1,[KEGG:00000],"[ACHE, ADCY5, ADCY6, ADCY7, ADCY8, ADCY9, AKT1...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
102,KEGG,KEGG:01522,Endocrine resistance,4.381889e-02,True,Endocrine resistance,65,318,8,9998,0.025157,0.123077,query_1,[KEGG:00000],"[ADCY5, ADCY6, ADCY7, ADCY8, ADCY9, AKT1, AKT2...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
103,KEGG,KEGG:05214,Glioma,4.650265e-02,True,Glioma,51,1478,19,9998,0.012855,0.372549,query_1,[KEGG:00000],"[AKT1, AKT2, AKT3, ARAF, BAK1, BAX, BRAF, CALM...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
104,KEGG,KEGG:05222,Small cell lung cancer,4.837540e-02,True,Small cell lung cancer,74,1822,26,9998,0.014270,0.351351,query_1,[KEGG:00000],"[AKT1, AKT2, AKT3, APAF1, BAK1, BAX, BCL2, BCL...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
